# Simulus Simple Use Cases

Simulus is a discrete-event simulator in Python. This document is to demonstrate how to run simulus via a few examples. This is not a tutorial. For that, try [Simulus Basics](simulus-basics.ipynb). All the examples shown in this guide can be found under the `examples/simple-tests` directory in the simulus source-code distribution.

Simulus uses [greenlet](https://pypi.org/project/greenlet/) to implement the process-oriented simulation world-view. Because of that, you need install greenlet using something like the following (you may need to run it in sudo):

In [ ]:
%%script bash

pip install greenlet

You'll also need to set PYTHONPATH to include the source directory. If you are running this jupyter notebook from the doc directory, you should find its parent directory in the PYTHONPATH:

In [ ]:
%env PYTHONPATH

If all are fine at this point, you can simply import the module 'simulus' to start using the simulator. 

In [ ]:
import simulus

### Use Case #1: Direct Event Scheduling

One can schedule functions to be executed at designated simulation time. The functions in this case are called event handlers (using the discrete-event simulation terminology).

In [ ]:
# %load "../examples/simple-case1.py"
import simulus

# An event handler for simulus is a python function with two parameters:
# the first parameter is the simulator, and the second parameters is
# the dictionary containing user-defined parameters for the event handler
def myfunc(sim, params):
    print(str(sim.now) + ": myfunc() runs with params=" + str(params))
    # schedule the next event at 5 seconds from now
    sim.sched(myfunc, params=params, offset=5)

# create an anonymous simulator
sim1 = simulus.simulator() 

# schedule the first event at 10 seconds
sim1.sched(myfunc, until=10, msg="hello world", value=100)

# advance simulation until 100 seconds
sim1.run(until=100)
print("simulator.run() ends at " + str(sim1.now))

# we can advance simulation for another 50 seconds
sim1.run(offset=50)
print("simulator.run() ends at " + str(sim1.now))


### Use Case #2: Simulation Process

A simulation process is an independent thread of execution. A process can be blocked and therefore advances its simulation time either by sleeping for some duration of time or by being blocked from synchronization primitives (such as semaphores).

In [ ]:
# %load "../examples/simple-case2.py"
import simulus

# A process for simulus is a python function with two parameters: 
# the first parameter is the simulator, and the second parameter is
# the dictionary containing user-defined parameters for the process
def myproc(sim, params):
    intv = params.get("intv")
    id = params.get("id")
    print(str(sim.now) + ": myproc(%d) runs with params=%r" % (id, params))
    while True:
        # suspend the process for some time
        sim.sleep(intv)
        print(str(sim.now) + ": myproc(%d) resumes execution" % id)

# create an anonymous simulator
sim2 = simulus.simulator()

# start a process 100 seconds from now
sim2.process(myproc, offset=100, intv=10, id=0)
# start another process 5 seconds from now
sim2.process(myproc, offset=5, intv=20, id=1)

# advance simulation until 200 seconds
sim2.run(until=200)
print("simulator.run() ends at " + str(sim2.now))

sim2.run(offset=50)
print("simulator.run() ends at " + str(sim2.now))


### Use Case #3: Process Synchronization with Semaphores

We illustrate the use of semaphore in the context of a classic producer-consumer problem. We are simulating a single-server queue (M/M/1) here.

In [ ]:
# %load "../examples/simple-case3.py"
import simulus

from random import seed, expovariate
from statistics import mean, median, stdev

# make it repeatable
seed(12345) 

# configuration of the single server queue: the mean inter-arrival
# time, and the mean service time
cfg = {"mean_iat":1, "mean_svc":0.8}

# keep the time of job arrivals, starting services, and departures
arrivals = []
starts = []
finishes = []

# the producer process waits for some random time from an 
# exponential distribution, and increments the semaphore 
# to represent a new item being produced, and then repeats 
def producer(sim, params):
    mean_iat = params.get("mean_iat")
    sem = params.get("sem")
    while True:
        iat = expovariate(1.0/mean_iat)
        sim.sleep(iat)
        print("%g: job arrives (iat=%g)" % (sim.now, iat))
        arrivals.append(sim.now)
        sem.signal()
        
# the consumer process waits for the semaphore (it decrements
# the value and blocks if the value is non-positive), waits for
# some random time from another exponential distribution, and
# then repeats
def consumer(sim, params):
    mean_svc = params.get("mean_svc")
    sem = params.get("sem")
    while True:
        sem.wait()
        print("%g: job starts service" % sim.now)
        starts.append(sim.now)
        svc = expovariate(1.0/mean_svc)
        sim.sleep(svc)
        print("%g: job departs (svc=%g)" % (sim.now, svc))
        finishes.append(sim.now)

# create an anonymous simulator
sim3 = simulus.simulator()

# create a semaphore with initial value of zero
sem = sim3.semaphore(0)

# start the producer and consumer processes
sim3.process(producer, params=cfg, sem=sem)
sim3.process(consumer, params=cfg, sem=sem)

# advance simulation until 100 seconds
sim3.run(until=100)
print("simulator.run() ends at " + str(sim3.now))

# calculate and output statistics
print(f'Results: jobs=arrival:{len(arrivals)}, starts:{len(starts)}, finishes:{len(finishes)}')
waits = [start - arrival for arrival, start in zip(arrivals, starts)]
totals = [finish - arrival for arrival, finish in zip(arrivals, finishes)]
print(f'Wait Time: mean={mean(waits):.1f}, stdev={stdev(waits):.1f}, median={median(waits):.1f}.  max={max(waits):.1f}')
print(f'Total Time: mean={mean(totals):.1f}, stdev={stdev(totals):.1f}, median={median(totals):.1f}.  max={max(totals):.1f}')
my_lambda = 1.0/cfg['mean_iat'] # mean arrival rate
my_mu = 1.0/cfg['mean_svc'] # mean service rate
my_rho = my_lambda/my_mu # server utilization
my_lq = my_rho*my_rho/(1-my_rho) # number in queue
my_wq = my_lq/my_lambda # wait in queue
my_w = my_wq+1/my_mu # wait in system
print(f'Theoretical Results: mean wait time = {my_wq:.1f}, mean total time = {my_w:.1f}')


### Use Case #4: Dynamic Processes

We continue with the previous example. At the time, rathar than using semaphores, we can achieve (exactly) the same results using simulus functions for dynamically creating processes.

In [ ]:
# %load "../examples/simple-case4.py"
import simulus

from random import seed, expovariate
from statistics import mean, median, stdev

# make it repeatable
seed(12345) 

# configuration of the single server queue: the mean inter-arrival
# time, and the mean service time
cfg = {"mean_iat":1, "mean_svc":0.8}

# keep the time of job arrivals, starting services, and departures
arrivals = []
starts = []
finishes = []

# we keep the account of the number of jobs in the system (those who
# have arrived but not yet departed); this is used to indicate whether
# there's a consumer process currently running; the value is more than
# 1, we don't need to create a new consumer process
jobs_in_system = 0

# the producer process waits for some random time from an exponential
# distribution to represent a new item being produced, creates a
# consumer process when necessary to represent the item being
# consumed, and then repeats
def producer(sim, params):
    global jobs_in_system
    mean_iat = cfg["mean_iat"]
    while True:
        iat = expovariate(1.0/mean_iat)
        sim.sleep(iat)
        print("%g: job arrives (iat=%g)" % (sim.now, iat))
        arrivals.append(sim.now)
        jobs_in_system += 1
        if jobs_in_system <= 1:
            sim.process(consumer)
        
# the consumer process waits for the semaphore (it decrements
# the value and blocks if the value is non-positive), waits for
# some random time from another exponential distribution, and
# then repeats
def consumer(sim, params):
    global jobs_in_system
    mean_svc = cfg["mean_svc"]
    while jobs_in_system > 0:
        print("%g: job starts service" % sim.now)
        starts.append(sim.now)
        svc = expovariate(1.0/mean_svc)
        sim.sleep(svc)
        print("%g: job departs (svc=%g)" % (sim.now, svc))
        finishes.append(sim.now)
        jobs_in_system -= 1

# create an anonymous simulator
sim3 = simulus.simulator()

# start the producer process only
sim3.process(producer)

# advance simulation until 100 seconds
sim3.run(until=100)
print("simulator.run() ends at " + str(sim3.now))

# calculate and output statistics
print(f'Results: jobs=arrival:{len(arrivals)}, starts:{len(starts)}, finishes:{len(finishes)}')
waits = [start - arrival for arrival, start in zip(arrivals, starts)]
totals = [finish - arrival for arrival, finish in zip(arrivals, finishes)]
print(f'Wait Time: mean={mean(waits):.1f}, stdev={stdev(waits):.1f}, median={median(waits):.1f}.  max={max(waits):.1f}')
print(f'Total Time: mean={mean(totals):.1f}, stdev={stdev(totals):.1f}, median={median(totals):.1f}.  max={max(totals):.1f}')
my_lambda = 1.0/cfg['mean_iat'] # mean arrival rate
my_mu = 1.0/cfg['mean_svc'] # mean service rate
my_rho = my_lambda/my_mu # server utilization
my_lq = my_rho*my_rho/(1-my_rho) # number in queue
my_wq = my_lq/my_lambda # wait in queue
my_w = my_wq+1/my_mu # wait in system
print(f'Theoretical Results: mean wait time = {my_wq:.1f}, mean total time = {my_w:.1f}')
